In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Accident_Anticipation/depth_estimation/DenseDepth/google_distance_code/google_dist_code_1/google-research
#%cd /content/gdrive/My\ Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/

# pip install --upgrade scikit-image
!set -e
!set -x

# !virtualenv -p python3 .
# !source ./bin/activate

# !pip install tensorflow
!pip install tensorflow-graphics
# !pip install -r depth_from_video_in_the_wild/requirements.txt

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import logging
import os
import image_utils
import cv2

import re
import time
import glob
from matplotlib import pyplot as plt

import numpy as np
from PIL import Image

import tensorflow as tf
from depth_from_video_in_the_wild import model

# https://github.com/google-research/google-research/tree/master/depth_from_video_in_the_wild

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Accident_Anticipation/depth_estimation/DenseDepth/google_distance_code/google_dist_code_1/google-research
     |████████████████████████████████| 245kB 5.0MB/s 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
"""A binary for generating odometry trajectories given a checkpoint."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import threading
import time

from absl import app
from absl import flags
from absl import logging
import numpy as np
import tensorflow as tf

from depth_from_video_in_the_wild import model_3
import cv2


WAIT_TIME = 20  # Wait time in seconds before checking for new checkpoint.
NUM_THREADS = 16  # Number of threads in which eval is calculated.

## generate new ODOMETRY_SETS that finds all folders in path
#
ODOMETRY_SETS = ['09-image_2', '10-image_2']

ImportError: ignored

In [3]:
flags.DEFINE_string('output_dir', None, 'Directory to store predictions. '
                    'Subdirectories will be created for each checkpoint.')
flags.DEFINE_string('odometry_test_set_dir', None,
                    'Directory where the odomotry test sets are.')
flags.DEFINE_string('checkpoint_path', None, 'Directory containing checkpoints '
                    'to evaluate.')
flags.DEFINE_integer('img_height', 128, 'Input frame height.')
flags.DEFINE_integer('img_width', 416, 'Input frame width.')


FLAGS = flags.FLAGS


def trajectory_inference():
  """Generates trajectories from the KITTI odometry sets and a checkpoint."""

  # Note that the struct2depth code only works at batch_size=1, because it uses
  # the training mode of batchnorm at inference.
  inference_model = model_3.Model(
      is_training=False,
      batch_size=1,
      img_height=FLAGS.img_height,
      img_width=FLAGS.img_width)
  saver = tf.train.Saver()
  sess = tf.Session()

  def infer_egomotion(image1, image2):
    return inference_model.inference_egomotion(image1, image2, sess)

  saver.restore(sess, FLAGS.checkpoint_path)
  if not tf.gfile.Exists(FLAGS.output_dir):
    tf.gfile.MakeDirs(FLAGS.output_dir)

  for odo_set in ODOMETRY_SETS:
    logging.info('Evaluating odometry on %s', odo_set)
    test_file_dir = os.path.join(FLAGS.odometry_test_set_dir, odo_set)
    output_file = os.path.join(FLAGS.output_dir, 'odometry_%s.txt' % odo_set)
    odometry_inference(test_file_dir, output_file, infer_egomotion)


def get_egomotion(im_files, results, infer_egomotion):
  for im_file in im_files:
    im = load_image(im_file)
    # Each image is a sequence of 3 frames. We use only the first 2.
    rot, trans = infer_egomotion(
        [im[:, :FLAGS.img_width, :]],
        [im[:, FLAGS.img_width:2 * FLAGS.img_width, :]])
    results[im_file] = (rot, trans)


def odometry_inference(image_sequence_dir, output_file, infer_egomotion):
  """Calculates egomotion inference and accumulates results into a trajectory.

  Args:
    image_sequence_dir: A string, directory where the odometry test sets reside.
    output_file: A string, file path where the trajectory is to be written.
    infer_egomotion: A callable that receives image1, image2 and outputs a
      rotation matrix and a translation vector connecting them in terms of
      egomotion.
  """
  im_files = sorted(tf.gfile.ListDirectory(image_sequence_dir))
  im_files = [
      os.path.join(image_sequence_dir, f)
      for f in im_files
      if 'png' in f and 'seg' not in f
  ]
  logging.info('Image Files:::')
  logging.info(im_files)
  num_images = len(im_files)

  # Divide the work to NUM_THREADS threads
  results = [None] * NUM_THREADS
  group_size = int(math.ceil(num_images / NUM_THREADS))
  threads = []
  for tid in range(NUM_THREADS):
    results[tid] = {}
    group_start = group_size * tid
    group_end = min(group_size * (tid + 1), num_images)
    im_group = im_files[group_start:group_end]
    threads.append(
        threading.Thread(
            target=get_egomotion,
            args=(im_group, results[tid], infer_egomotion)))

  def processed_images():
    return sum([len(results[th]) for th in range(NUM_THREADS)])

  threads.append(
      threading.Thread(
          target=_logger, args=(num_images, processed_images)))
  for th in threads:
    th.start()
  for th in threads:
    th.join()

  combined_results = {}
  for result in results:
    combined_results.update(result)

  # Accumulate the position and the orientation, to generate a trajectory.
  position = np.zeros(3)
  orientation = np.eye(3)
  with tf.gfile.Open(output_file, 'w') as f:
    logging.info('Writing results to %s', output_file)
    f.write('0.0 0.0 0.0\n')  # Initial position
    for im_file in sorted(im_files):
      rot, trans = combined_results[im_file]
      orientation = orientation.dot(rot[0])
      position += orientation.dot(trans[0])
      f.write(' '.join([str(p) for p in position]) + '\n')


def _logger(total_images, processed_images):
  """A helper function to log the progress of all eval threads."""
  p = 0
  prev_p = -1
  while p < total_images:
    p = processed_images()
    if p > prev_p:
      logging.info('Processed %d out of %d.', p, total_images)
      prev_p = p
    time.sleep(WAIT_TIME)


def load_image(img_file, resize=None, interpolation='linear'):
  """Load image from disk. Output value range: [0,1]."""
  with tf.gfile.Open(img_file, 'rb') as f:
    im_data = np.fromstring(f.read(), np.uint8)
  im = cv2.imdecode(im_data, cv2.IMREAD_COLOR)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  if resize and resize != im.shape[:2]:
    ip = cv2.INTER_LINEAR if interpolation == 'linear' else cv2.INTER_NEAREST
    im = cv2.resize(im, resize, interpolation=ip)
  return im.astype(np.float32) / 255.0


def main(_):
  trajectory_inference()


if __name__ == '__main__':
  app.run(main)

FATAL Flags parsing error: Unknown command line flag 'f'
Pass --helpshort or --helpfull to see help on flags.


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
def load_images(image_files):
    from skimage.transform import resize
    loaded_images = []
    iter1 = 0

    for file in image_files:
        crop_middle, crop_upper, crop_lower, img_shape = selective_cropping(file)

        if iter1 == 0:
          img_stack_middle, img_stack_upper, img_stack_lower = crop_middle, crop_upper, crop_lower
        elif img_stack_upper.any() != None:
          img_stack_middle, img_stack_upper, img_stack_lower = np.concatenate((img_stack_middle, crop_middle), axis = 0), np.concatenate((img_stack_upper, crop_upper), axis = 0), np.concatenate((img_stack_lower, crop_lower), axis = 0),
        else:
          img_stack_middle = np.concatenate((img_stack_middle, crop_middle), axis = 0)
        iter1+=1


    return img_stack_middle, img_stack_upper, img_stack_lower, img_shape


def crop(img):
    # Perform center cropping, preserving 50% vertically.
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[int(img.shape[0]*(half)):int(img.shape[0]*(1-half)), :]

    # Resize to match target height while preserving aspect ratio.
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))

    # Perform center cropping horizontally.
    remain = b.shape[1] - 416
    # cx /= (b.shape[1]/416)
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]

    return c

def crop_upper(img):
    # cropping the top
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[0:int(img.shape[0]*(middle_perc)), :]
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))
    remain = b.shape[1] - 416
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]
    return c

def crop_lower(img):
    # cropping the top
    middle_perc = 0.50
    left = 1-middle_perc
    half = left/2
    a = img[(-1)*int(img.shape[0]*(middle_perc))::, :]
    wdt = int((128*a.shape[1]/a.shape[0]))
    x_scaling = float(wdt)/a.shape[1]
    y_scaling = 128.0/a.shape[0]
    b = cv2.resize(a, (wdt, 128))
    remain = b.shape[1] - 416
    c = b[:, int(remain/2):b.shape[1]-int(remain/2)]
    return c

# random functions
def obtain_keys(lis):
    key_ls = list()
    for k in lis:
        integer = re.split('([0-9]+)', k)[1]
        key_ls.append(int(integer))
    return(key_ls)
  
def obtain_keys1(lis):
    key_ls = list()
    for k in lis:
        integer = re.split('([0-9]+)', k)[3]
        key_ls.append(int(integer))
    return(key_ls)
  
def h(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)
  
def sort_dir(lis):
    key_ls = h(obtain_keys(lis))
    return [lis[i] for i in key_ls]

## Perform cropping and resizing, place into network
def selective_cropping(img):
  x = np.clip(np.asarray(Image.open( img ), dtype=float) / 255, 0, 1)
  cropped = np.expand_dims(crop(x), axis = 0)
  cropped1 = None
  cropped2 = None
  if (np.abs(cropped.shape[1]-128)>30) and (np.abs(cropped.shape[2]-416)>30):
    x_1 = np.zeros((720,1280, 3))
    x_1[0:x.shape[0], 0:x.shape[1], :] = x  
    cropped = np.expand_dims(crop(x_1), axis = 0)
    print("Cropping pattern did not fit, resized.")
    print(img_folders)
  else:
    cropped1 = np.expand_dims(crop_upper(x), axis = 0)[:,:,0:-1,:]
    cropped2 = np.expand_dims(crop_lower(x), axis = 0)[:,:,0:-1,:]
  cropped = cropped[:,:,0:-1,:]
  img_shape = x.shape
  return cropped, cropped1, cropped2, img_shape

In [0]:
# tf.reset_default_graph() 
# train_model = model.Model(data_dir='depth_from_video_in_the_wild/data_example',is_training=True)
# saver  = train_model.saver

# # saver = tf.train.Saver()
sess = tf.Session()
# saver.restore(sess, save_path = 'depth_from_video_in_the_wild/depth_cp_2/model-1000977')

depth_model = model.Model(is_training=False, batch_size = 100)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Please use `layer.__call__` method instead.


INFO:tensorflow:Scale of 0 disables regularizer.


...

INFO:tensorflow:Scale of 0 disables regularizer.


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


INFO:tensorflow:Scale of 0 disables regularizer.


INFO:tensorflow:Scale of 0 disables regularizer.


In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
print(image_stack_middle.shape)
start_time = time.time()
output = depth_model.inference_depth(image_stack_middle, sess)
print(time.time() - start_time)

plt.imshow(output[0,:,:,0])

NameError: ignored

In [0]:

image_files = list()
image_files.append('./depth_from_video_in_the_wild/data_example/erfurt_93/0000000002.png')
# output_img = load_images(image_files)
# output_img[0].shape
# new_output = output_img[0][:,:,0:(416*2),:]
# new_output = np.concatenate((new_output,new_output,new_output,new_output),axis=0)
# image_tens = tf.convert_to_tensor(output_img)
# print(new_output.shape)

In [0]:
new_output = output[0][:,:,0:(416*2),:]
new_output = np.concatenate((new_output,new_output,new_output,new_output),axis=0)
print(new_output.shape)

NameError: ignored

In [0]:
!dir
# checkpoint path
# odometry test set dir
# output_dir

compile_protos.sh  depth_from_video_in_the_wild  lib	      README.md
CONTRIBUTING.md    image_utils.py		 LICENSE      third_party
depth_checkpoints  __init__.py			 __pycache__


In [0]:
print(os.listdir('./depth_from_video_in_the_wild/odometry_weights/odometry_learned_intrinsics/checkpoint/'))

## Getting pose from rotation matrix
## http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/MARBLE/high/high.htm

# http://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/MARBLE/high/pose/express.htm

['model-413174.index', 'model-413174.meta', 'model-413174.data-00000-of-00001']


In [0]:
!python -m depth_from_video_in_the_wild.trajectory_inference_1 \
!  --checkpoint_path=depth_from_video_in_the_wild/odometry_weights/odometry_learned_intrinsics/checkpoint/model-413174 \
!  --odometry_test_set_dir=depth_from_video_in_the_wild/data_example \
!  --output_dir=depth_from_video_in_the_wild/odometry_trajectory_outputs \
!  --alsologtostderr

# !python -m depth_from_video_in_the_wild.train \
# !  --data_dir=depth_from_video_in_the_wild/data_example \
# !  --checkpoint_dir=depth_from_video_in_the_wild/depth_cp_2 --train_steps=1

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

I1219 22:33:11.250159 139628325357440 model.py:86] batch_size: 1
I1219 22:33:11.250355 139628325357440 model.py:86] beta1: 0.9
I1219 22:33:11.250443 139628325357440 model.py:86] boxify: True
I1219 22:33:11.250509 139628325357440 model.py:86] data_dir: None
I1219 22:33:11.250575 139628325357440 model.py:86] depth_consistency_loss_weight: 0.01
I1219 22:33:11.250643 139628325357440 model.py:86] file_extensio

In [0]:
## odometry_inference
# 
im_files = sorted(tf.gfile.ListDirectory(image_sequence_dir))
im_files = [
    os.path.join(image_sequence_dir, f)
    for f in im_files
    if 'png' in f and 'seg' not in f
]
num_images = len(im_files)

# Divide the work to NUM_THREADS threads
results = [None] * NUM_THREADS
group_size = int(math.ceil(num_images / NUM_THREADS))
threads = []
for tid in range(NUM_THREADS):
  results[tid] = {}
  group_start = group_size * tid
  group_end = min(group_size * (tid + 1), num_images)
  im_group = im_files[group_start:group_end]
  threads.append(
      threading.Thread(
          target=get_egomotion,
          args=(im_group, results[tid], infer_egomotion)))

def processed_images():
  return sum([len(results[th]) for th in range(NUM_THREADS)])

threads.append(
    threading.Thread(
        target=_logger, args=(num_images, processed_images)))

In [0]:
str1 = ['training', 'testing']
str2 = ['positive','negative']

## three for loops, make outfile


for a1 in str1:
  for a2 in str2:
    
    root_path = '/content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/videos/frames/'
    dir_list = sort_dir(os.listdir(root_path+a1+'/'+a2+'/'))
    output_root = './outputs/'+a1+'/'+a2+'/'
    
    for a3 in dir_list:
      # Check which files were already created
      existing_list = sort_dir(os.listdir(output_root))
      curr_file = a3 + '.npz'
      
      if (curr_file in existing_list):
        pass
      
      else:
        print(a1, a2,a3)
        start_time = time.time()
        #print(a3)
        args_input = root_path+a1+'/'+a2+'/'+a3+'/*.jpg'
        args_output = output_root+a3+'.npz'
        files_list = glob.glob(args_input)
        start_time = time.time()

        sort_keys = h(obtain_keys1(files_list))
        r_files_list = [files_list[i] for i in sort_keys]
        
        print(r_files_list)

        # Input images
        output_list = load_images( r_files_list )
        inputs = output_list[0]
        x_shape = output_list[1]
        #print(inputs.shape)
        #print(x_shape)
        print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))

        # Compute results
        outputs = predict(model, inputs)

        outputs2 = scale_up(2,outputs)
        outputs_final = scale_up_2(x_shape[0],x_shape[1],outputs2)

        #### make code to save npy file in specified directory
        np.savez(args_output , distance=outputs_final)

        print('Saved file :' + str(args_output))

        print(time.time() - start_time)

print(a1)

testing
